In [80]:
from openseespy import opensees
import math
from streng.ppp.sections.geometry.tee import TeeSectionGeometry
from streng.ppp.sections.geometry.rectangular import RectangularSectionGeometry
from streng.ppp.opensees.post.convert_local_axes import os_to_sap_forces_beam, os_to_sap_forces_col

In [81]:
E = 30000000.
U = 0.0
G = E / (2.0*(1.0+U))

# Βάζω μειωτικό συντελεστή 0.5 στη δυσκαμψία της δοκόύ
tbeam = TeeSectionGeometry(bw=0.25, h=0.70, beff=1.20, hf=0.15)
beam_A = tbeam.bw * tbeam.h 
beam_I = tbeam.moment_of_inertia_xx * 0.5
beam_sheararea = tbeam.shear_area_2

# Χωρίς μειωτικό συντελεστή για τη δυσκαμψία του στύλου
col = RectangularSectionGeometry(b=0.40, h=0.40)
col_A = col.area
col_I = col.moment_of_inertia_xx
col_sheararea = col.shear_area_2

In [82]:
# General
opensees.wipe()
opensees.model('Basic', '-ndm', 2)
opensees.geomTransf('Linear', 1)

In [83]:
# Nodes
opensees.node(1, 0., 0.)
opensees.node(2, 5., 0.)
opensees.node(3, 0., 3.)
opensees.node(4, 5., 3.)

In [84]:
# Node fixes
opensees.fix(1, 1, 1, 1)
opensees.fix(2, 1, 1, 1)

In [85]:
# Nodal masses
opensees.mass(3, 11.723, 1.0e-10, 1.0e-10)

In [86]:
# Diaphragm - equalDOFs
opensees.equalDOF(3, 4, 1)

In [87]:
# ElasticTimoshenkoBeams
opensees.element('ElasticTimoshenkoBeam', 1, 1, 3, E, G, col_A, col_I, col_sheararea, 1) 
opensees.element('ElasticTimoshenkoBeam', 2, 2, 4, E, G, col_A, col_I, col_sheararea, 1) 
opensees.element('ElasticTimoshenkoBeam', 3, 3, 4, E, G, beam_A, beam_I, beam_sheararea, 1) 

In [88]:
# eigen_analysis
numEigen = 1
eigenValue = opensees.eigen('-fullGenLapack', numEigen)

lamb = eigenValue
period = 2 * math.pi / math.sqrt(lamb)
print(f'Period = {period:.4f}s')

Period = 0.1029s


In [89]:
# Gravity loads
opensees.timeSeries('Linear', 1)
opensees.pattern('Plain', 101, 1)

opensees.eleLoad('-ele', 3, '-type', '-beamUniform', -23.0)

In [90]:
opensees.system('BandGeneral')
opensees.constraints('Plain')
opensees.numberer('RCM')

opensees.algorithm('Linear')
opensees.integrator('LoadControl', 1.)
opensees.analysis('Static')
opensees.analyze(1)

0

In [93]:
print('Τα εντατικά μεγέθη μετατρέπονται σύμφωνα με τα πρόσημα του sap σε N1 V1 M1 N2 V2 M2')
print('element 1: ' + str(os_to_sap_forces_col([round(val, 2) for val in opensees.eleResponse(1, 'force')])) + ' - Στύλος αριστερά')
print('element 2: ' + str(os_to_sap_forces_col([round(val, 2) for val in opensees.eleResponse(2, 'force')])) + ' - Στύλος δεξιά')
print('element 3: ' + str(os_to_sap_forces_beam([round(val, 2) for val in opensees.eleResponse(3, 'force')])) + ' - Δοκός')

Τα εντατικά μεγέθη μετατρέπονται σύμφωνα με τα πρόσημα του sap σε N1 V1 M1 N2 V2 M2
element 1: [-57.5, -12.04, -11.78, -57.5, -12.04, 24.34] - Στύλος αριστερά
element 2: [-57.5, 12.04, 11.78, -57.5, 12.04, -24.34] - Στύλος δεξιά
element 3: [0.0, -57.5, -24.34, 0.0, 57.5, -24.34] - Δοκός


In [92]:
# opensees.wipe('all')